<a href="https://colab.research.google.com/github/skywalker00001/Conterfactual-Reasoning-Project/blob/main/samples_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# how to find out the different or changed blocks in sentence b compared to the sentence a (base)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
root = 'drive/MyDrive/LM/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sentencepiece
!pip install transformers -q

In [13]:
# Importing stock libraries
import numpy as np
import pandas as pd
import difflib
import nltk
import copy
import regex as re
import sys
import torch
sys.path.append('/content/drive/MyDrive/LM/')
from global_param import MyConfig
nltk.download("punkt")

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
myconfig = MyConfig()

In [5]:
PRETRAINED_MODEL_NAME = myconfig.PRETRAINED_MODEL_NAME
# tokenzier for encoding the text
t5_tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [11]:

input_ids = t5_tokenizer.encode_plus("The passenger walks in the park", max_length= myconfig.SOURCE_LEN, padding='max_length', return_tensors='pt')['input_ids'].squeeze()
print(input_ids)
print(len(input_ids))

tensor([   37, 10837, 10681,    16,     8,  2447,     1,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [21]:
input_ids = t5_tokenizer.encode_plus("The passenger walks in the park. I hate the movie.",return_tensors='pt')['input_ids'].squeeze()
print(input_ids)
print(len(input_ids))

tensor([   37, 10837, 10681,    16,     8,  2447,     5,    27,  5591,     8,
         1974,     5,     1])
13


In [31]:
input_ids = t5_tokenizer.encode_plus("The passenger <extra_id_2> in the park. I hate the movie.",return_tensors='pt')['input_ids'].squeeze()
print(input_ids)
print(len(input_ids))

tensor([   37, 10837, 32097,    16,     8,  2447,     5,    27,  5591,     8,
         1974,     5,     1])
13


In [29]:
test_ids = torch.tensor([5])
output = t5_tokenizer.decode(input_ids, skip_special_tokens=False)
print(output)

The passenger<extra_id_1> in the park. I hate the movie.</s>


In [ ]:
input_ids = t5_tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids

In [2]:
'''
  Compared the changes between origin to edited setnence.
  For example:  Input:
      origin = "I want to acchieve something to school."
      edited = "I do something to go for it to school."
  Output: 
      "1-2:\"do something\".3-5:\"go for it\"."
'''

def origin2edited(origin, edited):
    origin_list = nltk.word_tokenize(origin)
    editeed_list = nltk.word_tokenize(edited)
    s = difflib.SequenceMatcher(None, origin_list, editeed_list)
    cgs = []
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if (tag != 'equal'):
            cgs.append(f'{i1}-{i2}:\"{(" ").join(editeed_list[j1:j2])}\".')
    conclusion = "".join(cgs)
    return conclusion

In [3]:
'''
    Restore the edited sentence based on the changes cg and the original sentence.
    For example:  Input:
        cg = 1-2:\"do something\".3-5:\"go for it\".
        origin = "I want to acchieve something to school."
    Output:
        "I do something to go for it to school."
'''

def restore(cg, origin):
    cgs = cg.split('.')[0:-1]
    origin_list = nltk.word_tokenize(origin)
    #blocks = copy.deepcopy(origin)
    for j in list(reversed(cgs)):
        #print("unchanged: ", origin_list)
        #print(j)
        pattern = re.compile(r'^(\d+)-(\d+):\"(.*?)\"')  # matches 1-2:"do something".  (1), (2), (do something)
        results = re.search(pattern, j)  # i3 means the content between ""
        i1, i2, i3 = results.group(1), results.group(2), results.group(3)
        origin_list[int(i1): int(i2)] = nltk.word_tokenize(i3)
        #print("changed: ", origin_list)
    return (" ").join(origin_list[0:-1])+'.'  #because we don't want any space before '.', which is the last element in the origin_list

In [5]:
# testing
a = "I want to acchieve something to school."
b = "I do something to go for it to school."
# a_list = nltk.word_tokenize(a)
# b_list = nltk.word_tokenize(b)

# a_list = "qabxcd"
# b_list = "abycdf"
# print(a_list)
# print(b_list)


In [6]:
print(origin2edited(a,b))
print(restore(origin2edited(a,b), a))

1-2:"do something".3-5:"go for it".
I do something to go for it to school.


# 1. Preparation

In [7]:
#training df
small_path = root + '/TimeTravel/train_supervised_small.json'
small_df = pd.read_json(small_path, lines=True)
small_df.head()
print(len(small_df))

16752


In [8]:
# Data cleaning, remove the examples with the same original_ending and edited_ending

comb_small_text=[]

for i in range(len(small_df)):

  comb_small_text.append(small_df.loc[i, 'edited_ending'][0] +" "+ small_df.loc[i, 'edited_ending'][1] +" "+ \
                small_df.loc[i, 'edited_ending'][2])
small_df.insert(6, "ground_truth", comb_small_text)
small_cleaned_df = small_df[small_df.ground_truth != small_df.original_ending]
small_cleaned_df = small_cleaned_df.reset_index() 
print(len(small_cleaned_df))

15798


In [9]:
# text_a: source, text_b: target, 
text_a, text_b, word_changes = [], [], []

for i in range(len(small_cleaned_df)):
  ori_text = small_cleaned_df.loc[i, 'original_ending']
  text_a.append("premise: " + small_cleaned_df.loc[i, 'premise'] + " initial: " + \
                small_cleaned_df.loc[i, 'initial'] + " counterfactual: " + small_cleaned_df.loc[i, 'counterfactual'] + \
                " original_ending: " + ori_text)
  #text_a.append(re.sub(re_pat, df.loc[i, 'edit1'], df.loc[i, 'original1']))
  edited_text = small_cleaned_df.loc[i, 'edited_ending'][0] +" "+ small_cleaned_df.loc[i, 'edited_ending'][1] +" "+ \
                small_cleaned_df.loc[i, 'edited_ending'][2]
  text_b.append("edited_ending: " + edited_text)
  word_change = origin2edited(ori_text, edited_text)
  word_changes.append(word_change)

In [10]:
train_df = pd.DataFrame({'source_text': text_a, 'target_text': text_b, 'word_changes': word_changes}) 
train_df.head()

,source_text,target_text,word_changes
0,premise: On my way to work I stopped to get so...,edited_ending: I paid the cashier and patientl...,"7-7:""at the counter""."
1,premise: Terry aspired to be a chef. initial: ...,edited_ending: He decided he would continue th...,"16-17:""customers"".19-21:""made"".23-23:""profitab..."
2,premise: Kim and her glass went on a field tri...,edited_ending: Instead of going home they went...,"0-2:""Instead of going home""."
3,premise: Susie was sitting on her barstool. in...,"edited_ending: Suddenly, an earthquake sent he...","2-4:""an earthquake""."
4,premise: Celeste rode her motorcycle across th...,edited_ending: She didn't have enough time to ...,"12-14:""deer ,""."


In [11]:
#train_df.to_excel(root + 'TimeTravel/' + 'output' + model_version + '.xlsx')
train_df.to_excel(root + 'TimeTravel/' + 'cleaned_small_1.0.xlsx')

# trash code

In [ ]:

# a = "I paid the cashier and patiently waited for my drink."
# b = "I paid the cashier and patiently waited at the counter for my drink."
a = "I want to acchieve something to school."
b = "I do something to go for it to school."
a_list = nltk.word_tokenize(a)
b_list = nltk.word_tokenize(b)

a_list = "qabxcd"
b_list = "abycdf"
print(a_list)
print(b_list)

qabxcd
abycdf


In [ ]:
s = difflib.SequenceMatcher(None, a_list, b_list)
for block in s.get_matching_blocks():
    print(block)
    

Match(a=1, b=0, size=2)
Match(a=4, b=3, size=2)
Match(a=6, b=6, size=0)


In [ ]:
print(a_list[2])

b


In [ ]:
print(a_list[0:0+8])
print(a_list[8:8+3])
print(a_list[11:11+0])

qabxcd




In [ ]:
matches = []
matches.append([0, 0, 0])
for block in s.get_matching_blocks():
    #matches.append([block[0], block[1], block[2]])
    matches.append([i for i in block])
    #matches.append(block)
print(matches)   

# explanation: matches[i][0] are the a index, matches[i][1] are the b index, matches[i] [2] are the lengths of same (matched) words.

[[0, 0, 0], [1, 0, 2], [4, 3, 2], [6, 6, 0]]


In [ ]:
changes = []
for i in range(len(matches) - 1):
    #print(matches[i])
    if ((matches[i][0]+ matches[i][2] < matches[i+1][0]) & (matches[i][1]+ matches[i][2] < matches[i+1][1])):     # replace
        string = (" ").join(b_list[(matches[i][1]+matches[i][2]) : matches[i+1][1]])
        changes.append(f"replacing_{matches[i][0]+matches[i][2]}-{matches[i+1][0]}:\"{string}\".")
    elif ((matches[i][0]+ matches[i][2] < matches[i+1][0]) & (matches[i][1]+ matches[i][2] == matches[i+1][1])):     # delete
        string = (" ").join(b_list[(matches[i][1]+matches[i][2]) : matches[i+1][1]])
        changes.append(f"deleting_{matches[i][0]+matches[i][2]}-{matches[i+1][0]}:\"{string}\".")
    elif ((matches[i][0]+ matches[i][2] == matches[i+1][0]) & (matches[i][1]+ matches[i][2] < matches[i+1][1])):     # insert
        string = (" ").join(b_list[(matches[i][1]+matches[i][2]) : matches[i+1][1]])
        changes.append(f"inserting_{matches[i][0]+matches[i][2]}-{matches[i+1][0]}:\"{string}\".")

In [ ]:
print(changes)

['deleting_0-1:"".', 'replacing_3-4:"y".', 'inserting_6-6:"f".']


In [ ]:
c = copy.deepcopy(a_list)
c[7:7] = nltk.word_tokenize("at the counter")


TypeError: ignored

In [ ]:
print(a_list)

['I', 'want', 'to', 'acchieve', 'something', 'to', 'school', '.']


In [ ]:
print(c)

['I', 'want', 'to', 'acchieve', 'something', 'to', 'school', 'at', 'the', 'counter', '.']


# another way

In [ ]:
# a = "qabxcd"
# b = "abycdf"
cgs = []
s = difflib.SequenceMatcher(None, a_list, b_list)
for tag, i1, i2, j1, j2 in s.get_opcodes():
    print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    #print('{:7}   a[{}:{}] --> b[{}:{}] {!r} --> {!r}'.format(tag, i1, i2, j1, j2, a[i1:i2], b[j1:j2]))
    if (tag != 'equal'):
        #cgs.append(f'{tag}_{i1}-{i2}:\"{(" ").join(b_list[j1:j2])}\".')
        cgs.append(f{i1}-{i2}:\"{(" ").join(b_list[j1:j2])}\".')

delete    a[0:1] --> b[0:0]      'q' --> ''
equal     a[1:3] --> b[0:2]     'ab' --> 'ab'
replace   a[3:4] --> b[2:3]      'x' --> 'y'
equal     a[4:6] --> b[3:5]     'cd' --> 'cd'
insert    a[6:6] --> b[5:6]       '' --> 'f'


In [ ]:
print(cgs)

['0-1:"".', '3-4:"y".', '6-6:"f".']


In [ ]:
t=(" ".join(cgs))
print(t)

0-1:"". 3-4:"y". 6-6:"f".


In [ ]:
print(t.split(' '))

['0-1:"".', '3-4:"y".', '6-6:"f".']


function

In [ ]:
def origin2edited(origin, edited):
    origin_list = nltk.word_tokenize(origin)
    editeed_list = nltk.word_tokenize(edited)
    s = difflib.SequenceMatcher(None, origin_list, editeed_list)
    cgs = []
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if (tag != 'equal'):
            cgs.append(f'{i1}-{i2}:\"{(" ").join(editeed_list[j1:j2])}\".')
    conclusion = "".join(cgs)
    return conclusion

In [ ]:
print(origin2edited(a, b))


1-2:"do something".3-5:"go for it".


In [ ]:
cgs = sen.split('.')[0:-1]
print(len(cgs))

2


In [ ]:
def restore(cg, origin):
    cgs = cg.split('.')[0:-1]
    origin_list = nltk.word_tokenize(origin)
    #blocks = copy.deepcopy(origin)
    for j in list(reversed(cgs)):
        #print("unchanged: ", origin_list)
        #print(j)
        pattern = re.compile(r'^(\d+)-(\d+):\"(.*?)\"')  # matches 1-2:"do something".  (1), (2), (do something)
        results = re.search(pattern, j)  # i3 means the content between ""
        i1, i2, i3 = results.group(1), results.group(2), results.group(3)
        origin_list[int(i1): int(i2)] = nltk.word_tokenize(i3)
        #print("changed: ", origin_list)
    return (" ").join(origin_list[0:-1])+'.'  #because we don't want any space before '.', which is the last element in the origin_list

In [ ]:
print(a)

I want to acchieve something to school.


In [ ]:
print(sen)

1-2:"do something".3-5:"go for it".


In [ ]:
b = "I do something to go for it to school."

In [ ]:
print(restore(origin2edited(a, b), a))

unchanged:  ['I', 'want', 'to', 'acchieve', 'something', 'to', 'school', '.']
3-5:"go for it"
changed:  ['I', 'want', 'to', 'go', 'for', 'it', 'to', 'school', '.']
unchanged:  ['I', 'want', 'to', 'go', 'for', 'it', 'to', 'school', '.']
1-2:"do something"
changed:  ['I', 'do', 'something', 'to', 'go', 'for', 'it', 'to', 'school', '.']
I do something to go for it to school.


In [ ]:
i1 = re.compile(r'^(\d+)-(\d+):\"(.*?)\"')
#i1 = re.compile(r'(\d+)')
sen = "1-2:\"do something\".3-5:\"go for it\"."
i3 = re.search(i1, sen).group(1)
print(i3)

1


In [ ]:
pattern = re.compile(r'^(\d+)-(\d+):\"(.*?)\"')  # matches 1-2:"do something".  (1), (2), (do something)
results = re.search(pattern, sen)  # i3 means the content between ""
i1, i2, i3 = results.group(1), results.group(2), results.group(3)
print(i1, i2, i3)

1 2 do something


In [ ]:
print(restore(orgin2edited(a, b), a))

TypeError: ignored